<a href="https://colab.research.google.com/github/CatherineMacfarlane/collab-test/blob/main/Catherine_rag_pipeline_ara_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Importing Haystack package
%%bash
pip install haystack-ai
pip install "datasets>=2.6.1"
pip install "sentence-transformers>=2.2.0"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.2/345.2 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.1/324.1 kB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 837.8/837.8 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 57.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 263.8/263.8 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.6 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of lxml[html-clean] to determine which version is compatible wit

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.


In [ ]:
# Importing all relevant packages for this pipeline
from pathlib import Path
import os

from datasets import load_dataset
from haystack import Document, Pipeline
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack.components.embedders import SentenceTransformersDocumentEmbedder, SentenceTransformersTextEmbedder
from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever
from haystack.components.builders import PromptBuilder
from haystack.components.generators import HuggingFaceAPIGenerator
from haystack.utils.auth import Secret
from huggingface_hub import snapshot_download


In [ ]:
#Creating a document store
document_store = InMemoryDocumentStore()

In [ ]:
#Loading data for ofsted
dataset = load_dataset("csv", data_files = "/content/drive/MyDrive/Catherine Shared Management /DSA/2024-05-23-labeled_ara_dataset.csv")

In [ ]:
dataset["train"].train_test_split(test_size=0.1)

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'alb number', 'ara_name', 'department_acroynm', 'department_name', 'alb_acroynm', 'alb_name', 'classification', 'document_type', 'fy_year', 'link', 'style', 'section_header', 'content', 'Original_Page_ID', 'text_id', 'character_length', 'string_length', 'staff_turnover', 'sickness_absense', 'employee_engagement', 'information_technology', 'fte', 'statement_of_financial_position'],
        num_rows: 22177
    })
    test: Dataset({
        features: ['Unnamed: 0', 'alb number', 'ara_name', 'department_acroynm', 'department_name', 'alb_acroynm', 'alb_name', 'classification', 'document_type', 'fy_year', 'link', 'style', 'section_header', 'content', 'Original_Page_ID', 'text_id', 'character_length', 'string_length', 'staff_turnover', 'sickness_absense', 'employee_engagement', 'information_technology', 'fte', 'statement_of_financial_position'],
        num_rows: 2465
    })
})

In [ ]:
dataset


DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'alb number', 'ara_name', 'department_acroynm', 'department_name', 'alb_acroynm', 'alb_name', 'classification', 'document_type', 'fy_year', 'link', 'style', 'section_header', 'content', 'Original_Page_ID', 'text_id', 'character_length', 'string_length', 'staff_turnover', 'sickness_absense', 'employee_engagement', 'information_technology', 'fte', 'statement_of_financial_position'],
        num_rows: 24642
    })
})

In [ ]:
[print(doc) for doc in dataset[:5]]

KeyError: "Invalid key: slice(None, 5, None). Please first select a split. For example: `my_dataset_dictionary['train'][slice(None, 5, None)]`. Available splits: ['train']"

In [ ]:
docs = [Document(content=doc["content"]) for doc in dataset]

TypeError: string indices must be integers

In [ ]:
doc_embedder = SentenceTransformersDocumentEmbedder(model="sentence-transformers/all-MiniLM-L6-v2")
doc_embedder.warm_up()

In [ ]:
docs_with_embeddings = doc_embedder.run(docs)
document_store.write_documents(docs_with_embeddings["documents"])